In [ ]:
# Importando a biblioteca

import pandas as pd
import numpy as np
import sqlite3
import gdown

**1. EXTRAÇÃO**

In [ ]:
# Carregamento inicial dos dados brutos
# ID do arquivo
arquivo_id = "1A4MVsMqteVw2aNvIAGNvwciOs6rEMMkv"

# Monta a URL para o gdown
url = f"https://drive.google.com/uc?id={arquivo_id}"
output = "netflix.csv"

gdown.download(url, output, quiet=False)

try:
 df_netflix_titles = pd.read_csv("netflix.csv")
 print("Arquivo extraído com sucesso!")

 print("Visualização das 5 primeiras linhas do dataset:")

except FileNotFoundError:
 print(f"Erro: Arquivo não encontrado")

df_netflix_titles.head()

Downloading...
From: https://drive.google.com/uc?id=1A4MVsMqteVw2aNvIAGNvwciOs6rEMMkv
To: /content/netflix.csv
100%|██████████| 3.40M/3.40M [00:00<00:00, 156MB/s]


Arquivo extraído com sucesso!
Visualização das 5 primeiras linhas do dataset:


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
# Exibição de tipos de colunas
df_netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
print("\Contagem de calores nulos por Coluna:")
print(df_netflix_titles.isnull().sum())

\Contagem de calores nulos por Coluna:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
/tmp/ipython-input-901989378.py:1: SyntaxWarning: invalid escape sequence '\C'
  print("\Contagem de calores nulos por Coluna:")


In [ ]:
print("Visualizando linhas com dados faltantes:")
#Apenas linhas que tem pelo menos um campo vazio
print(df_netflix_titles[df_netflix_titles.isnull().any(axis=1)].head())

Visualizando linhas com dados faltantes:
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2

In [ ]:
print("\Estatísticas")
print(df_netflix_titles['release_year'].describe())

\Estatísticas
count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: release_year, dtype: float64


<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
/tmp/ipython-input-3023070729.py:1: SyntaxWarning: invalid escape sequence '\E'
  print("\Estatísticas")


**2. TRANSFORMAÇÃO**

In [ ]:
# Criada uma cópia para a camada silver

df_silver = df_netflix_titles.copy()

In [ ]:
# Limpeza Inicial

# Removidas duplicatas
df_silver.drop_duplicates(inplace=True)

In [ ]:
# Para validar se há duplicidades
numero_de_duplicatas_restantes = df_silver.duplicated().sum()
if numero_de_duplicatas_restantes == 0:
    print("✅ Sucesso! Todas as duplicatas foram removidas.")
else:
    print(f"⚠️ Atenção! Ainda existem {numero_de_duplicatas_restantes} duplicatas.")

✅ Sucesso! Todas as duplicatas foram removidas.


In [ ]:
# Tratamento de nulos: Preencher com 'Desconhecido'
dicionario = {
    'director': 'Diretor Desconhecido',
    'cast': 'Elenco Desconhecido',
    'country': 'País Desconhecido'
}

df_silver.fillna(dicionario, inplace=True)

In [ ]:
# Para validar se há nulos

# Lista das colunas que foram tratadas
colunas_tratadas = ['director', 'cast', 'country']

# Série com a contagem de nulos nas colunas tratadas
contagem_nulos = df_silver[colunas_tratadas].isnull().sum()
print("Contagem de nulos restantes")
print(contagem_nulos)
print("---------------------------------------")


total_nulos_restantes = contagem_nulos.sum()
if total_nulos_restantes == 0:
    print("✅ Sucesso! Não há mais valores nulos nas colunas 'director', 'cast' e 'country'.")
else:
    print(f"⚠️ Atenção! Ainda existem {total_nulos_restantes} valores nulos nessas colunas. Verifique a execução anterior.")

Contagem de nulos restantes
director    0
cast        0
country     0
dtype: int64
---------------------------------------
✅ Sucesso! Não há mais valores nulos nas colunas 'director', 'cast' e 'country'.


In [ ]:
# Removidos espaços em branco no início e fim de todas as colunas de texto
for col in df_silver.select_dtypes(include='object').columns:
    df_silver[col] = df_silver[col].str.strip()

In [ ]:
# Validar espaços em branco nos textos

# Identificar as colunas de texto (object)
colunas_texto = df_silver.select_dtypes(include='object').columns
total_problemas = 0
resultados_por_coluna = {}

print("Verificação de espaços que restaram")


for col in colunas_texto:
    # Checar se há valores que começam ou terminam com espaço
    tem_espaco_inicio = df_silver[col].astype(str).str.startswith(' ').sum()
    tem_espaco_fim = df_silver[col].astype(str).str.endswith(' ').sum()

    total_coluna = tem_espaco_inicio + tem_espaco_fim

    if total_coluna > 0:
        resultados_por_coluna[col] = total_coluna

    total_problemas += total_coluna

# Resultados
if total_problemas == 0:
    print("✅ Sucesso! Nenhum espaço em branco encontrado no início ou fim das colunas de texto.")
else:
    print(f"⚠️ Atenção! Encontrados {total_problemas} problemas de espaços em branco remanescentes nas seguintes colunas:")
    for col, count in resultados_por_coluna.items():
        print(f"  - Coluna '{col}': {count} ocorrências de espaço no início/fim.")

Verificação de espaços que restaram
✅ Sucesso! Nenhum espaço em branco encontrado no início ou fim das colunas de texto.


In [ ]:
#Padronizados nomes para Primeira Maiúscula

colunas_titulos = ['title', 'director', 'cast']
for col in colunas_titulos:
    df_silver[col] = df_silver[col].str.title()

In [ ]:
colunas_titulos = ['title', 'director', 'cast']
total_inconformidades = 0
inconformidades_por_coluna = {}

print("Verificação de Inconformidades")

for col in colunas_titulos:
    # Checa se o valor original é diferente do valor após a re-aplicação do .str.title()
    # Se forem diferentes, o valor original (após a sua execução) não estava no formato correto.
    inconforme = (df_silver[col] != df_silver[col].str.title())
    count = inconforme.sum()

    if count > 0:
        inconformidades_por_coluna[col] = count
        total_inconformidades += count

if total_inconformidades == 0:
    print("✅ Sucesso! Os valores nas colunas 'title', 'director' e 'cast' estão padronizados no formato Title Case (Primeira Maiúscula).")
else:
    print(f"⚠️ Atenção! Encontradas {total_inconformidades} inconformidades no formato Title Case nas seguintes colunas:")
    for col, count in inconformidades_por_coluna.items():
        print(f"  - Coluna '{col}': {count} valores fora do padrão esperado.")

Verificação de Inconformidades
✅ Sucesso! Os valores nas colunas 'title', 'director' e 'cast' estão padronizados no formato Title Case (Primeira Maiúscula).


In [ ]:
# Dicionário de unificação dos países
map_paises = {
    'united states': 'United States',
    'u.s.a.': 'United States',
    'usa': 'United States',
    'brasil': 'Brazil',
    'brazil': 'Brazil',
    'uk': 'United Kingdom',
    'u.k.': 'United Kingdom',
    'england': 'United Kingdom',
    'south korea': 'Korea, Republic of',
    'republic of korea': 'Korea, Republic of'
}

# Aplicação da unificação
df_silver['country'] = df_silver['country'].str.lower().replace(map_paises)

In [ ]:
# Dicionário de Classificação Etária para padrão Brasil
# Livre, 10+, 12+, 14+, 16+, 18+
classificacao = {
        'TV-Y': 'Livre', 'TV-G': 'Livre', 'G': 'Livre', 'TV-Y7': '10+',
        'TV-Y7-FV': '10+', 'PG': '10+', 'PG-13': '12+', 'TV-14': '14+',
    'R': '16+', 'TV-MA': '18+', 'NR': 'Não Classificado', 'UR': 'Não Classificado'
}

df_silver['rating_br'] = df_silver['rating'].map(classificacao).fillna('Outros')

In [ ]:
# Ver se a nova coluna 'rating_br'tem 6 classificações
print("Distribuição da Classificação Etária (rating_br)")
print(df_silver['rating_br'].value_counts(dropna=False))

print("\n--------------------------------------------------------------")
print("✅ Verifique se o resultado inclui apenas as 6 classificações BR (Livre, 10+, 12+, 14+, 16+, 18+), 'Não Classificado' e 'Outros'.")

Distribuição da Classificação Etária (rating_br)
rating_br
18+                 3207
14+                 2160
Outros               873
16+                  799
10+                  627
Livre                568
12+                  490
Não Classificado      83
Name: count, dtype: int64

--------------------------------------------------------------
✅ Verifique se o resultado inclui apenas as 6 classificações BR (Livre, 10+, 12+, 14+, 16+, 18+), 'Não Classificado' e 'Outros'.


In [ ]:
# Ajuste datas e tipos
#Convertidas date_added para datetime
df_silver['date_added'] = pd.to_datetime(df_silver['date_added'], format='mixed', errors='coerce')

In [ ]:
# Checa o tipo de dado da coluna 'date_added'
dtype_atual = df_silver['date_added'].dtype

if dtype_atual == 'datetime64[ns]':
    print("✅ Sucesso! A coluna 'date_added' foi convertida para o tipo datetime (datetime64[ns]).")
else:
    print(f"⚠️ Atenção! A coluna não é do tipo datetime. O dtype encontrado é: {dtype_atual}")

✅ Sucesso! A coluna 'date_added' foi convertida para o tipo datetime (datetime64[ns]).


In [ ]:
#Criadas colunas de Ano e Mês de adição
df_silver['ano_adicionado'] = df_silver['date_added'].dt.year
df_silver['mes_adicionado'] = df_silver['date_added'].dt.month_name()

In [ ]:
df_silver.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,rating_br,ano_adicionado,mes_adicionado
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Elenco Desconhecido,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",12+,2021.0,September
1,s2,TV Show,Blood & Water,Diretor Desconhecido,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",south africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",18+,2021.0,September
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",país desconhecido,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,18+,2021.0,September
3,s4,TV Show,Jailbirds New Orleans,Diretor Desconhecido,Elenco Desconhecido,país desconhecido,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",18+,2021.0,September
4,s5,TV Show,Kota Factory,Diretor Desconhecido,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",india,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,18+,2021.0,September


In [ ]:
# Calculado intervalo entre o lançamento e a adição na plataforma
df_silver['lancamento_adicao'] = df_silver['ano_adicionado'] - df_silver['release_year']

In [ ]:
# Colunas para conferir
colunas_teste = ['release_year', 'ano_adicionado', 'lancamento_adicao']

# Seleciona 5 linhas aleatórias
amostra_calculo = df_silver[colunas_teste].sample(5)

print("\n--- Amostra da Lógica de Cálculo (5 Linhas) ---")
print(amostra_calculo)

print("\n--- Confirmação ---")
print("✅ Verifique visualmente na amostra se: 'ano_adicionado' - 'release_year' = 'lancamento_adicao'.")


--- Amostra da Lógica de Cálculo (5 Linhas) ---
      release_year  ano_adicionado  lancamento_adicao
8007          2017          2018.0                1.0
6804          2009          2020.0               11.0
4552          1979          2018.0               39.0
2413          2020          2020.0                0.0
1939          2014          2020.0                6.0

--- Confirmação ---
✅ Verifique visualmente na amostra se: 'ano_adicionado' - 'release_year' = 'lancamento_adicao'.


In [ ]:
# Duração - Separar valor númerico da unidade (min vs seasons)
df_silver[['duration_valor', 'duration_unidade']] = df_silver['duration'].str.split(' ', expand=True)
df_silver['duration_valor'] = pd.to_numeric(df_silver['duration_valor'], errors='coerce')

In [ ]:
# Cria uma coluna de resumo da duração
df_silver['duracao_resumo'] = df_silver['duration_valor'].astype(str) + ' ' + df_silver['duration_unidade']

print("--- Amostra da Coluna 'duracao_resumo' ---")
print(df_silver[['title', 'type', 'duracao_resumo']].sample(5))

--- Amostra da Coluna 'duracao_resumo' ---
                            title   type duracao_resumo
2434                  Let'S Dance  Movie      116.0 min
7492  Monster High: Freaky Fusion  Movie       74.0 min
1882      Dolly Parton: Here I Am  Movie       90.0 min
8786                 You Can Tutu  Movie       87.0 min
2277                  Madras Café  Movie      128.0 min


In [ ]:
# Converter para Inteiro INT
df_silver['duration_valor'] = df_silver['duration_valor'].fillna(0).astype(int)

In [ ]:
# Amostra visual
colunas_teste = ['duration', 'duration_valor', 'duration_unidade']

# Seleciona 5 linhas aleatórias para verificação
amostra_split = df_silver[colunas_teste].sample(5)

print("\n--- Amostra do Split e Conversão (5 Linhas) ---")
print(amostra_split)

print("\n--- Confirmação ---")
print("✅ Verifique visualmente se a coluna 'duration' foi dividida corretamente entre 'duration_valor' (número) e 'duration_unidade' (min/seasons).")


--- Amostra do Split e Conversão (5 Linhas) ---
     duration  duration_valor duration_unidade
8708   94 min              94              min
4736  131 min             131              min
1074   79 min              79              min
4232   56 min              56              min
5919   72 min              72              min

--- Confirmação ---
✅ Verifique visualmente se a coluna 'duration' foi dividida corretamente entre 'duration_valor' (número) e 'duration_unidade' (min/seasons).


In [ ]:
# Novas Colunas

# Gênero principal (primeiro da lista)
df_silver['genero_principal'] = df_silver['listed_in'].str.split(',').str[0].str.strip()

#País principal (primeiro da lista)
df_silver['pais_principal'] = df_silver['country'].str.split(',').str[0].str.strip()

In [ ]:
# Tabela dimensão para 'genero_principal'
generos_unicos = df_silver['genero_principal'].unique()
df_dim_generos = pd.DataFrame(generos_unicos, columns=['nome_genero'])

# Criação do ID único (chave primária)
df_dim_generos.reset_index(inplace=True)
df_dim_generos.rename(columns={'index': 'id_genero_principal'}, inplace=True)

# Começa o ID em 1 para ser mais amigável
df_dim_generos['id_genero_principal'] = df_dim_generos['id_genero_principal'] + 1

# Chave Estrangeira
mapa_genero_para_id = df_dim_generos.set_index('nome_genero')['id_genero_principal'].to_dict()

# Adiciona a chave estrangeira na tabela principal (df_silver)
df_silver['id_genero_principal'] = df_silver['genero_principal'].map(mapa_genero_para_id)

print("\n--- Tabela de Gêneros (Amostra) ---")
print(df_dim_generos.head())
print("\n--- Chave Estrangeira adicionaada à Tabela principal (Amostra) ---")
print(df_silver[['show_id', 'genero_principal', 'id_genero_principal']].head())


--- Tabela de Gêneros (Amostra) ---
   id_genero_principal             nome_genero
0                    1           Documentaries
1                    2  International TV Shows
2                    3          Crime TV Shows
3                    4              Docuseries
4                    5               TV Dramas

--- Chave Estrangeira adicionaada à Tabela principal (Amostra) ---
  show_id        genero_principal  id_genero_principal
0      s1           Documentaries                    1
1      s2  International TV Shows                    2
2      s3          Crime TV Shows                    3
3      s4              Docuseries                    4
4      s5  International TV Shows                    2


In [ ]:
# Tabela de dimensão para 'pais_principal'
paises_unicos = df_silver['pais_principal'].unique()
df_dim_paises = pd.DataFrame(paises_unicos, columns=['nome_pais'])

# ID único (chave primária)
df_dim_paises.reset_index(inplace=True)
df_dim_paises.rename(columns={'index': 'id_pais_principal'}, inplace=True)

# Começa o ID em 1
df_dim_paises['id_pais_principal'] = df_dim_paises['id_pais_principal'] + 1

# Chave Estrangeira
mapa_pais_para_id = df_dim_paises.set_index('nome_pais')['id_pais_principal'].to_dict()

# Adiciona a chave estrangeira na tabela principal (df_silver)
df_silver['id_pais_principal'] = df_silver['pais_principal'].map(mapa_pais_para_id)

print("\n--- Tabela de Países (Amostra) ---")
print(df_dim_paises.head())
print("\n--- Chave Estrangeira adicionada à Tabela principal (Amostra) ---")
print(df_silver[['show_id', 'pais_principal', 'id_pais_principal']].head())


--- Tabela de Países (Amostra) ---
   id_pais_principal          nome_pais
0                  1      United States
1                  2       south africa
2                  3  país desconhecido
3                  4              india
4                  5      united states

--- Chave Estrangeira adicionada à Tabela principal (Amostra) ---
  show_id     pais_principal  id_pais_principal
0      s1      United States                  1
1      s2       south africa                  2
2      s3  país desconhecido                  3
3      s4  país desconhecido                  3
4      s5              india                  4


In [ ]:
# Colunas para conferir
colunas_teste = ['listed_in', 'genero_principal', 'country', 'pais_principal']

# Seleciona 5 linhas aleatórias para verificação
amostra_split = df_silver[colunas_teste].sample(5)

print("--- Amostra da separação (5 Linhas) ---")
print(amostra_split)

print("\n--- Confirmação ---")
print("✅ Verifique visualmente se:")
print("1. 'genero_principal' é o primeiro valor de 'listed_in' (antes da vírgula).")
print("2. 'pais_principal' é o primeiro valor de 'country' (antes da vírgula).")

--- Amostra da separação (5 Linhas) ---
                                              listed_in    genero_principal  \
6998  British TV Shows, Docuseries, International TV...    British TV Shows   
4253     Dramas, International Movies, Sci-Fi & Fantasy              Dramas   
3114      Dramas, International Movies, Romantic Movies              Dramas   
2132                                 Action & Adventure  Action & Adventure   
2202               Comedies, Dramas, Independent Movies            Comedies   

             country  pais_principal  
6998  united kingdom  united kingdom  
4253   United States   United States  
3114         nigeria         nigeria  
2132   United States   United States  
2202   United States   United States  

--- Confirmação ---
✅ Verifique visualmente se:
1. 'genero_principal' é o primeiro valor de 'listed_in' (antes da vírgula).
2. 'pais_principal' é o primeiro valor de 'country' (antes da vírgula).


In [ ]:
# Gênero Principal
print("\n--- Top 5 Gêneros Principais ---")
print(df_silver['genero_principal'].value_counts().head())

# País Principal
print("\n--- Top 5 Países Principais ---")
print(df_silver['pais_principal'].value_counts().head())

print("\n--- Confirmação ---")
print("✅ Os valores mais frequentes em 'genero_principal' e 'pais_principal' parecem limpos e corretos.")


--- Top 5 Gêneros Principais ---
genero_principal
Dramas                    1600
Comedies                  1210
Action & Adventure         859
Documentaries              829
International TV Shows     774
Name: count, dtype: int64

--- Top 5 Países Principais ---
pais_principal
United States        2818
india                1008
país desconhecido     831
united kingdom        628
united states         393
Name: count, dtype: int64

--- Confirmação ---
✅ Os valores mais frequentes em 'genero_principal' e 'pais_principal' parecem limpos e corretos.


**3. MODELAGEM E CARGA**

In [ ]:
# Dicionário de tradução das colunas (Inglês -> Português)
rename_map = {
    # Colunas originais
    'show_id': 'id_show', 'type': 'tipo', 'title': 'título', 'director': 'diretor',
    'cast': 'elenco', 'country': 'país', 'date_added': 'data_adicionada',
    'release_year': 'ano_lancamento', 'rating': 'classificacao_indicativa',
    'duration': 'duração', 'listed_in': 'generos', 'description': 'descricao',

    # Colunas derivadas (tradução ou padronização)
    'rating_br': 'classificacao_br',
    'ano_adicionado': 'ano_adicionado',
    'mes_adicionado': 'mes_adicionado',
    'lancamento_adicao': 'tempo_adicao_plataforma',
    'duration_valor': 'duracao_valor',
    'duration_unidade': 'duracao_unidade',
    'duracao_resumo': 'duracao_resumo',
    'proporcao_catalogo': 'proporcao_catalogo',
    'genero_principal': 'genero_principal',
    'pais_principal': 'pais_principal',

    # CHAVES ESTRANGEIRAS ADICIONADAS:
    'id_genero_principal': 'id_genero_principal',
    'id_pais_principal': 'id_pais_principal'
}

# Verificação
print("✅ Colunas do DataFrame 'df_netflix_titles' traduzidas com sucesso.")
print("\nNovas colunas:")
print(df_netflix_titles.columns.tolist())

✅ Colunas do DataFrame 'df_netflix_titles' traduzidas com sucesso.

Novas colunas:
['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [ ]:
# Criadas tabelas auxiliares para analise 'um para muitos'
# Tabela Fato: Títulos (Dados principais consolidados - Camada Gold)
# Criada a cópia, após a criação de TODAS as colunas de valor (proporção, duração, etc.)
df_gold_titulos = df_silver.copy()

df_gold_titulos.rename(columns=rename_map, inplace=True) # Tradução

In [ ]:
print("\n--- Amostra das primeiras 5 Linhas (df_gold_titulos) ---")
print(df_gold_titulos.head())

print("\n--- Confirmação ---")
print("✅ Os dados parecem ter sido copiados integralmente para a nova tabela 'df_gold_titulos'.")


--- Amostra das primeiras 5 Linhas (df_gold_titulos) ---
  id_show     tipo                 título               diretor  \
0      s1    Movie   Dick Johnson Is Dead       Kirsten Johnson   
1      s2  TV Show          Blood & Water  Diretor Desconhecido   
2      s3  TV Show              Ganglands       Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans  Diretor Desconhecido   
4      s5  TV Show           Kota Factory  Diretor Desconhecido   

                                              elenco               país  \
0                                Elenco Desconhecido      United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...       south africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...  país desconhecido   
3                                Elenco Desconhecido  país desconhecido   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...              india   

  data_adicionada  ano_lancamento classificacao_indicativa    duração  ...  \
0      202

In [ ]:
#Tabela Auxiliar: Gêneros
df_gold_generos = df_silver[['show_id', 'listed_in']].copy()
df_gold_generos['genero'] = df_gold_generos['listed_in'].str.split(',')
df_gold_generos = df_gold_generos.explode('genero')
df_gold_generos['genero'] = df_gold_generos['genero'].str.strip()
df_gold_generos.rename(columns={'show_id': 'id_show'}, inplace=True)
df_gold_generos = df_gold_generos[['id_show', 'genero']]

In [ ]:
#Tabela Auxiliar: Países
df_gold_paises = df_silver[['show_id', 'country']].copy()
df_gold_paises['paises'] = df_gold_paises['country'].str.split(',')
df_gold_paises = df_gold_paises.explode('paises')
df_gold_paises['paises'] = df_gold_paises['paises'].str.strip()
df_gold_paises.rename(columns={'show_id': 'id_show'}, inplace=True)
df_gold_paises = df_gold_paises[['id_show', 'paises']]

In [ ]:
# Teste para a Tabela Auxiliar df_gold_paises

linhas_silver = df_silver.shape[0]
linhas_gold_paises = df_gold_paises.shape[0]

print("--- Verificação paises (df_gold_paises) ---")
print(f"Linhas em df_silver: {linhas_silver}")
print(f"Linhas em df_gold_paises: {linhas_gold_paises}")

if linhas_gold_paises >= linhas_silver:
    print("✅ Sucesso! O número de linhas é maior ou igual, confirmando que a operação explode funcionou.")
else:
    print("⚠️ Atenção! O número de linhas diminuiu, indicando um erro na criação da tabela.")

# Colunas e limpeza da última camada
print("\n--- Colunas Finais de df_gold_paises ---")
print(df_gold_paises.columns.tolist())

# Checa a limpeza da coluna 'paises' (Top 10)
print("\n--- Top 10 Países---")
print(df_gold_paises['paises'].value_counts().head(10))

# 5 Linhas
print("\n--- Amostra da Tabela df_gold_paises (5 Linhas) ---")
print(df_gold_paises[['id_show', 'paises']].sample(5))

print("\n--- Confirmação ---")
print("✅ Verifique se as colunas são apenas ['id_show', 'paises'] e se os nomes dos países estão limpos (sem espaços no início/fim).")

--- Verificação paises (df_gold_paises) ---
Linhas em df_silver: 8807
Linhas em df_gold_paises: 11049
✅ Sucesso! O número de linhas é maior ou igual, confirmando que a operação explode funcionou.

--- Colunas Finais de df_gold_paises ---
['id_show', 'paises']

--- Top 10 Países---
paises
United States        2818
india                1046
united states         872
país desconhecido     831
united kingdom        806
canada                445
france                393
japan                 318
spain                 232
germany               226
Name: count, dtype: int64

--- Amostra da Tabela df_gold_paises (5 Linhas) ---
     id_show         paises
7785   s7786         france
4784   s4785         france
6475   s6476  United States
1696   s1697         canada
1746   s1747         canada

--- Confirmação ---
✅ Verifique se as colunas são apenas ['id_show', 'paises'] e se os nomes dos países estão limpos (sem espaços no início/fim).


In [ ]:
# Cópia da coluna de elenco
df_cast = df_silver['cast'].copy()

# Divide a string de elenco por vírgula em uma lista
df_cast = df_cast.str.split(',')

# Cada item da lista para uma nova linha
df_cast = df_cast.explode()

# Limpa espaços em branco
df_cast = df_cast.str.strip()

# Conta a frequência de cada ator
contagem_atores = df_cast.value_counts()

print("--- Top 10 atores ---")
print(contagem_atores.head(10))

--- Top 10 atores ---
cast
Elenco Desconhecido    825
Anupam Kher             43
Shah Rukh Khan          35
Julie Tejwani           33
Naseeruddin Shah        32
Takahiro Sakurai        32
Rupa Bhimani            31
Om Puri                 30
Akshay Kumar            30
Yuki Kaji               29
Name: count, dtype: int64


In [ ]:
# Cria uma cópia da coluna de diretores
df_director = df_silver['director'].copy()

# Divide a string de diretor por vírgula em uma lista
df_director = df_director.str.split(',')

# Cada item da lista para uma nova linha
df_director = df_director.explode()

# Limpa espaços em branco
df_director = df_director.str.strip()

# Conta a frequência de cada diretor
contagem_diretores = df_director.value_counts()

print("\n--- Top 10 Diretores ---")
print(contagem_diretores.head(10))


--- Top 10 Diretores ---
director
Diretor Desconhecido    2634
Rajiv Chilaka             22
Jan Suter                 21
Raúl Campos               19
Marcus Raboy              16
Suhas Kadav               16
Jay Karas                 15
Cathy Garcia-Molina       13
Martin Scorsese           12
Youssef Chahine           12
Name: count, dtype: int64


In [ ]:
# Contar o número total de filmes e séries
proporcao = df_silver['type'].value_counts()

# Calcula a porcentagem
total_itens = proporcao.sum()
proporcao_percentual = (proporcao / total_itens) * 100

print("--- Proporção de Filmes e Séries no Catálogo ---")
print(proporcao)
print("\n--- Proporção Percentual (%) ---")
print(proporcao_percentual.round(2))

--- Proporção de Filmes e Séries no Catálogo ---
type
Movie      6131
TV Show    2676
Name: count, dtype: int64

--- Proporção Percentual (%) ---
type
Movie      69.62
TV Show    30.38
Name: count, dtype: float64


In [ ]:
# Total de cada tipo
contagem_tipo = df_silver['type'].value_counts()
total = contagem_tipo.sum()

# Nova coluna com a proporção percentual
def calcular_proporcao(row):
    tipo = row['type']
    contagem = contagem_tipo[tipo]
    return f"{((contagem / total) * 100):.2f}% de {tipo}"

df_silver['proporcao_catalogo'] = df_silver.apply(calcular_proporcao, axis=1)

print("\n--- Amostra da Coluna 'proporcao_catalogo' ---")
print(df_silver[['title', 'type', 'proporcao_catalogo']].head())


--- Amostra da Coluna 'proporcao_catalogo' ---
                   title     type proporcao_catalogo
0   Dick Johnson Is Dead    Movie    69.62% de Movie
1          Blood & Water  TV Show  30.38% de TV Show
2              Ganglands  TV Show  30.38% de TV Show
3  Jailbirds New Orleans  TV Show  30.38% de TV Show
4           Kota Factory  TV Show  30.38% de TV Show


In [ ]:
# Nova tabela - Contagem de títulos por ano de lançamento
df_contagem_por_ano = df_silver.groupby('release_year')['show_id'].count().reset_index()
df_contagem_por_ano.columns = ['ano_lancamento', 'total_titulos']

print("--- Tabela de Resumo: Títulos por ano de lançamento ---")
print(df_contagem_por_ano.sort_values(by='ano_lancamento', ascending=False).head(10))

print("\n✅ Tabela 'df_contagem_por_ano' criada com sucesso.")

--- Tabela de Resumo: Títulos por ano de lançamento ---
    ano_lancamento  total_titulos
73            2021            592
72            2020            953
71            2019           1030
70            2018           1147
69            2017           1032
68            2016            902
67            2015            560
66            2014            352
65            2013            288
64            2012            237

✅ Tabela 'df_contagem_por_ano' criada com sucesso.


In [ ]:
# CONEXÃO COM BANCO DE DADOS LOCAL (SQLite)
conn = sqlite3.connect('projeto_netflix.db')

# Salvando as tabelas no banco de dados
try:
    df_gold_titulos.to_sql('dim_titulos', conn, if_exists='replace', index=False)
    df_gold_generos.to_sql('fato_generos', conn, if_exists='replace', index=False)
    df_gold_paises.to_sql('fato_paises', conn, if_exists='replace', index=False)
    df_dim_generos.to_sql('dim_generos', conn, if_exists='replace', index=False)
    df_dim_paises.to_sql('dim_paises', conn, if_exists='replace', index=False)
    df_gold_generos.to_sql('ponte_generos_todos', conn, if_exists='replace', index=False)
    df_gold_paises.to_sql('ponte_paises_todos', conn, if_exists='replace', index=False)

    print("Carga realizada com sucesso! Tabelas criadas")

except Exception as e:
    print(f'Erro na carga: {e}')
finally:
    conn.close()

print('\n--- Pipeline Finalizado ---')
print("O arquivo 'projeto_netflix.db' está pronto para ser conectado ao Power Bi")

Carga realizada com sucesso! Tabelas criadas

--- Pipeline Finalizado ---
O arquivo 'projeto_netflix.db' está pronto para ser conectado ao Power Bi
